# Full algorithm - generating grocery lists

1. Define variables: days-times-stores
2. Define limits: min amount-max amount
3. Generate (random) lists from narrowed down df

### Import libraries 

In [44]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dataframe_image as dfi
from datetime import time
import matplotlib.dates as mdates
from matplotlib.ticker import StrMethodFormatter
from matplotlib.pyplot import figure
from random import sample

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)

class bcolors:
    WARNING = '\033[91m'
    BOLD = '\033[1m'

In [45]:
    # Function to generate #visits per day of the week
from AlgorithmDAYS_HH2 import beforeNY, AfterNY
    # Function to generate store name (&type) per weekday (+ assign #of visits per day)
from AlgorithmSTORE_HH2 import Mondays, Tuesdays, Wednesdays, Thursdays, Fridays, Saturdays, Sundays
    #Function to define the limits for total visits, days, same store and type per week
from AlgorithmCOUNTS_HH2 import CountTotalVisits, CountTotalDays, CountStoreName, CountStoreType, CountTotalPerday, CountVisitsPerDay
    # Function to decide time (morning, noon. afternoon) per visited store
from AlgorithmTIMES_HH2 import AlbertHeijn, Okay, Delhaize, Carrefour, VersavelPoelman, Sys, Brabo, Ikea, Kruidvat
    # Function to decide amounts per visit & per week
from AlgorithmAMOUNT_HH2 import StoreAmount, WeekAmount, StoreCount, WeekCount

In [46]:
dataframe = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv")

# Define variables - day/time/store

## Days & visits

In [47]:
CountTotalVisits()

(4, 8)

In [48]:
CountTotalDays()

(2, 5)

In [49]:
i = 0

while i < 2:

    ### 1. What DAY?

        # Select the relevant df for this generated grocery list, comment out the other one
    # df = beforeNY()
    df = AfterNY()

        # identify counts for visits per weekday
    mo = int(df["times"].values[0])
    tu = int(df["times"].values[1])
    we = int (df["times"].values[2])
    th = int (df["times"].values[3])
    fr = int (df["times"].values[4])
    sa = int (df["times"].values[5])
    su = int (df["times"].values[6])
    
        
        ### CHECKPOINT: range for visits/week and days/week
        
        # Check if the total GROCERY VISITS PER WEEK are within the normal range
    visitcount = df['times'].sum()
    totalvisits = CountTotalVisits()
    minvisits = totalvisits[0]
    maxvisits = totalvisits[1]
    if minvisits <= visitcount <= maxvisits:
        i= i+1
    else:
        print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few/many visits per week")
        i=0
        continue

        # Check if the total GROCERY DAYS PER WEEK are within the normal range
    dayscount = df['times'].nunique()
    totaldays = CountTotalDays()
    mindays = totaldays[0]
    maxdays = totaldays[1]
    if mindays <= dayscount <= maxdays:
        i = i+1
        display(df)
    else:
        print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few/many days per week")
        i=0
        continue

 RERUN - Too few/many visits per week
 RERUN - Too few/many visits per week


,times,day
4,2.0,Monday
6,0.0,Tuesday
4,0.0,Wednesday
6,1.0,Thursday
4,0.0,Friday
6,1.0,Saturday
4,0.0,Sunday


## Stores & time

In [50]:
restart = True 

while restart:

    ### 2. What STORE?

        # Generate store names for the [previously identified] shopping dags + corresponding store type
    stores = [Mondays(mo), Tuesdays(tu), Wednesdays(we), Thursdays(th), Fridays(fr), Saturdays(sa), Sundays(su)]
        # create one df for all visited stores + day of the week
    stores = pd.concat(stores)


    ### 3. What TIME?

        # Identifier counts for store names per week
    AHt = (stores['store_name'] == 'Albert Heijn').sum()
    OKt = (stores['store_name'] == 'Okay').sum()
    DHt = (stores['store_name'] == 'Delhaize').sum()
    CFt = (stores['store_name'] == 'Carrefour').sum()
    VPt = (stores['store_name'] == 'Versavel Poelman').sum()
    SYt = (stores['store_name'] == 'Sys').sum()
    BRt = (stores['store_name'] == 'Brabo').sum()
    IKt = (stores['store_name'] == 'Ikea').sum()
    KRt = (stores['store_name'] == 'Kruidvat').sum()

        # Generate times per storename and compile in df with stores
    storetimes = [AlbertHeijn(AHt), Okay(OKt), Delhaize(DHt), Carrefour(CFt), VersavelPoelman(VPt), Sys(SYt), Brabo(BRt), Ikea(IKt), Kruidvat(KRt)]
    storetimes = pd.concat(storetimes)
    stores = pd.merge(stores, storetimes, on="store_name")

    ### CHECKPOINT: range for amount of stores per week.day

        # Check if the total STORE TYPES VISITED PER WEEK are within the normal range
    for storetype in dataframe['store_type']:
        typecount = stores[stores['store_type']==storetype]['store_type'].count()
        countstoretype = CountStoreType(storetype)
        minstoretype = countstoretype[0]
        maxstoretype = countstoretype[1]
        if typecount < minstoretype:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few store type:", storetype)
            restart = True
            break # force restart
        elif maxstoretype < typecount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many store type:", storetype)
            restart = True
            break # force restart
        else:
            pass


        # Check if the total STORE NAMES VISITED PER WEEK are within the normal range
    for storename in dataframe['store_name']:
        namecount = stores[stores['store_name']==storename]['store_name'].count()
        countstorename = CountStoreName(storename)
        minstorename = countstorename[0]
        maxstorename = countstorename[1]
        if namecount < minstorename:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few store name:", storename)
            restart = True
            break # force restart
        elif maxstorename < namecount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many store name:", storename)
            restart = True
            break # force restart
        else:
            pass


        # Check if the total STORES VISITED PER DAY are within the normal range
    for day in dataframe['day']:
        perdaycount = stores[stores['day']==day]['day'].count()
        CountTotalPerday = CountVisitsPerDay(day)
        minstoreperday = CountTotalPerday[0]
        maxstoreperday = CountTotalPerday[1]
        if namecount < minstoreperday:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few stores on:", day)
            restart = True
            break # force restart
        elif maxstoreperday < namecount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many stores on:", day)
            restart = True
            break # force restart
        else:
            restart = False
            display(stores)
            break

 RERUN - Too many store type: supermarket
 RERUN - Too many store name: Albert Heijn
 RERUN - Too many stores on: Saturday


,store_name,store_type,day,time
0,Versavel Poelman,butcher,Monday,morning
1,Sys,bakery,Monday,morning
2,Okay,supermarket,Thursday,afternoon
3,Carrefour,supermarket,Saturday,afternoon


## Define limits per week (item counts)

In [51]:
WeekCount()

(31, 98)

In [87]:
storetime = stores[['store_name','time']]
storetime

restart = True 

while restart:
    storecounts = []

    for store, time in storetime.itertuples(index=False):
        storecounts.append(StoreCount(store,time))

    stores['counts'] = storecounts
    
    # Check if the total ITEMS PER STORE are within the normal range
    for items in dataframe['item_type']:
        week9count = stores['counts'].sum()
        NormWeekCount = WeekCount()
        minWeekCount = NormWeekCount[0]
        maxWeekCount = NormWeekCount[1]
        if week9count < minWeekCount:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too few items:", week9count)
            restart = True
            break # force restart
        elif maxWeekCount < week9count:
            print(f"{bcolors.WARNING}{bcolors.BOLD} RERUN - Too many items:", week9count)
            restart = True
            break # force restart
        else:
            restart = False
            display(stores)
            break

,store_name,store_type,day,time,counts
0,Versavel Poelman,butcher,Monday,morning,6
1,Sys,bakery,Monday,morning,1
2,Okay,supermarket,Thursday,afternoon,23
3,Carrefour,supermarket,Saturday,afternoon,31


# The Grocery Lists

In [121]:
    ## Create dataframes for each day they go shopping
typedf = {}

for storetype in stores['store_type']:
        storeday = dataframe[dataframe['store_type'] == storetype]
        items = storeday[['item_type', 'item_name', 'day', 'store_type','store_name', 'amount']]
        typedf[storetype] = pd.DataFrame(items)
        typedf[storetype].drop_duplicates()


    ## Split up the dataframes to store name for mon, tue, sat, sun
storetypes = stores.store_type.values.tolist()
days = stores.day.values.tolist()           
thisdaysdf = {}
daydf = {}

if 'supermarket'in stores.values:
        for day in days:
            thisdaydf = typedf['supermarket']
            storesdf = thisdaydf[thisdaydf['day'] == day]
            items = storesdf[['item_type', 'item_name', 'store_type', 'store_name', 'amount']]
            daydf[day] = pd.DataFrame(items)
            daydf[day].drop_duplicates()

## Sample the dataframes per day they go shopping
    ## function for supermarket item lists
def SupermarketItems():
        dayamount = stores[['store_type', 'day', 'store_name', 'counts']]
        if 'supermarket' in dayamount.values:
                grocerylist = dayamount[dayamount['store_type'] == 'supermarket']
                dayslist = grocerylist.day.values.tolist()
                amounts = grocerylist.counts.values.tolist()
                for amount, day in zip(amounts, dayslist):
                    if 'supermarket' in dayamount.values:
                        daydf[day]= daydf[day].drop_duplicates()
                        return (daydf[day].sample(n=amount, replace=False))
                    else:
                        pass

    ## function for other store types' item lists
def StoretypeItems(storetype):                   
        if storetype in dayamount.values:
                grocerylist = dayamount[dayamount['store_type'] == storetype]
                for amount in grocerylist['counts']:
                    if storetype in dayamount.values:
                        typedf[storetype] = typedf[storetype].drop_duplicates()
                        typedf[storetype] = typedf[storetype].drop(columns=['day'])
                        return(typedf[storetype].sample(n=amount, replace=False))
                    else:
                        pass
                    
    ## run both functions
for storetype in stores['store_type']:
        if storetype != 'supermarket':
            display(StoretypeItems(storetype))
        
n = stores['store_type'].value_counts()['supermarket']

for _ in range(n):
        display(SupermarketItems()) 

,item_type,item_name,store_type,store_name,amount
283,pork,HOEVE BURGER,butcher,Versavel Poelman,1
129,charcuterie,boerenring,butcher,Brabo,1
207,charcuterie,GEB. BOULETTEN,butcher,Versavel Poelman,1
130,charcuterie,chateaubriand,butcher,Brabo,1
41,charcuterie,DROGE WORST,butcher,Versavel Poelman,1
38,fish spread,NEPTUNESALADA,butcher,Versavel Poelman,1
284,meat spread,PREPARE DU CHEF,butcher,Versavel Poelman,1


,item_type,item_name,store_type,store_name,amount
279,bread,GROF BROOD GESN.,bakery,Sys,1


,item_type,item_name,store_type,store_name,amount
331,chocolate,MILKA TABLET,supermarket,Albert Heijn,1
118,andalouse sauce,BONI saus andalouse TD 420ml,supermarket,Okay,1
112,french fries,BELVIVA pom'pin 750g,supermarket,Okay,1
109,chicken,"OKAY gebakken kip met saus 1,1kg",supermarket,Okay,1
120,spaghetti,BONI spaghetti volkoren 500g,supermarket,Okay,1
153,pita bread,Boni pitabroodjes voorgebakken 6st 480g,supermarket,Okay,1
119,pasta sauce,SPAGHERONI pastasaus tradizionale 520g,supermarket,Okay,1
121,coffee,BONI Lungo cups UTZ 16st,supermarket,Okay,1
154,iced tea,BONI Green Tea niet-bruisend 33cl,supermarket,Okay,6
108,celeriac,knolselder 1stuk,supermarket,Okay,1


,item_type,item_name,store_type,store_name,amount
370,lunch salad,AH MLTSALADE,supermarket,Albert Heijn,1
333,fruit salad,FRUITSALADE,supermarket,Albert Heijn,1
120,spaghetti,BONI spaghetti volkoren 500g,supermarket,Okay,1
122,face masks,wegwerp mondmaskers volwassenen 5st,supermarket,Okay,1
111,vegetable mix,FLORETTE groentenmix Bordje Krokant 250g,supermarket,Okay,1
126,bell peppers,paprika punt zoet rood Los,supermarket,Okay,1
121,coffee,BONI Lungo cups UTZ 16st,supermarket,Okay,1
369,mints,MENTOS,supermarket,Albert Heijn,1
105,cheese,BONI gouda light sneetjes 250g,supermarket,Okay,1
115,apple sauce,EVERYDAY appelcompote stuk 710g,supermarket,Okay,1
